## Figure 3c,d,e

In [1]:
# Preliminaries to work with the data.   
%matplotlib inline
%run __init__.py
import subprocess
from utils import loading, scoring
from gerkin import dream,params,fit2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Get the current git hash to save/load models for this version of the code.  
git_hash = subprocess.getoutput("git log --pretty=format:'%h' -n 1")

In [3]:
# Load the data
sets = ['training','leaderboard','testset']
descriptors = loading.get_descriptors(format='True')
all_CIDs = loading.get_CIDs(sets)
all_CID_dilutions = loading.get_CID_dilutions(sets)
#features = dream.get_molecular_data(['dragon','episuite','morgan','nspdk','gramian'],all_CIDs)
features = loading.get_molecular_data(['dragon','morgan'],all_CIDs)

Dragon has 4869 features for 476 molecules.
Morgan has 2437 features for 476 molecules.
There are now 7306 total features.


In [ ]:
# Create the feature and descriptor arrays 
X_rf = dream.make_X(features,all_CID_dilutions)[0]
X_lin = dream.quad_prep(features,all_CID_dilutions)
Y = loading.load_perceptual_data(sets)
Y = dream.filter_Y_dilutions(Y,concentration='all') # Average over replicates

The X matrix now has shape (952x5519) molecules by non-NaN good molecular descriptors
The X matrix now has shape (952x14614) molecules by non-NaN good molecular descriptors


### Load or compute the random forest model

In [ ]:
# Load or compute the random forest model correlations (obtained from cross-validation)
use_saved_rf_model = False #  Change this to a git hash to use a corresponding pre-computed model,
                           # to 'Old' to use the exact model used in the paper, 
                           # or to False to compute the model.  
n_splits = 5#100
n_estimators = 50
n_features_list = [1,2,3,4,5,10,33,100,333,1000,3333,10000]
if use_saved_rf_model == 'Old': # Legacy
    rs0 = np.load('../../data/forest_rs_250_0.npy')
    rs0_9 = np.load('../../data/forest_rs_250_0-9.npy')
    rs10_20 = np.load('../../data/forest_rs_250_10-20.npy')
    rs_forest = rs0_9.copy()
    rs_forest[0,:,:] = rs0[0,:,:]
    rs_forest[10:,:,:] = rs10_20[10:,:,:]
    rs_forest = np.ma.array(rs_forest,mask=np.isnan(rs_forest))
elif type(use_saved_rf_model) is str:
    rs_forest = np.load('../../data/rs_forest_%d_%s.npy' % (n_estimators,use_saved_rf_model))
else:
    trans_params = params.get_trans_params(Y, descriptors, plot=False)
    use_et, max_features, max_depth, min_samples_leaf, trans_weight, regularize, use_mask = params.get_other_params()
    rs_forest = fit2.feature_sweep(X_rf,Y,n_estimators=n_estimators,n_splits=n_splits,
                                   n_features_list=n_features_list,model='rf',rfe=False,
                                   max_features=max_features,max_depth=max_depth,min_samples_leaf=min_samples_leaf)
    np.save('../../data/rs_forest_%d_%s' % (n_estimators,git_hash),rs_forest.data)

[                       1%                       ] 2 out of 105 complete (Fitting descriptor #0, split #2)

### Load or compute the linear model

In [ ]:
# Load or compute the feature ranks for the training set
use_saved_linear_feature_ranks = False
n_splits = 5#25
if use_saved_linear_feature_ranks == 'Old':
    lin_ranked = np.load('../../data/lin_ranked_rick.npy')
elif type(use_saved_linear_feature_ranks) is str:
    lin_ranked = np.load('../../data/lin_ranked_%s.npy' % use_saved_linear_feature_ranks)
else:
    # -1 is to leave out the leak feature
    lin_ranked = fit2.compute_linear_feature_ranks_cv(X_lin,Y,n_resampling=10,n_splits=n_splits)
    np.save('../../data/lin_ranked_%s' % git_hash,lin_ranked)

In [ ]:
use_saved_lin_model = False
n_splits = 5#100
if use_saved_lin_model == 'Old':
    rs_lin = np.load('../../data/rs_lin_rick_right.npy')
    rs_lin = np.ma.array(rs_lin,mask=np.isnan(rs_lin))
elif type(use_saved_lin_model) is str:
    rs_lin = np.load('../../data/rs_lin_%s.npy' % use_saved_lin_model)
else:
    rs_lin = fit2.feature_sweep(X_lin,Y,n_splits=n_splits,model='ridge',rfe=False,
                                n_features_list=n_features_list,alpha=10.0,lin_ranked=lin_ranked)
    np.save('../../data/rs_lin_%s' % git_hash,rs_lin.data)

## Figure 3c

In [ ]:
from scipy.interpolate import interp1d

# Plotting function
def plot(forest,lin,n_features_list,labels=['forest','quadratic'],threshold=0.8):
    fig,axes = plt.subplots(3,7,sharex=False,sharey=True,figsize=(20,10))
    thresholds, thresh_locs = np.zeros(21), np.zeros(21)
    n_features_list_new = np.logspace(0,4,1000)
    for col in range(21):
        curve = forest[col,:,:].mean(axis=1)
        f = interp1d(n_features_list,curve)
        curve = f(n_features_list_new)
        thresholds[col] = max(curve[0],threshold*curve.max())
        thresh_index = np.where(curve>thresholds[col])[0][0]
        #print(col,thresh_index,len(n_features2),n_features2[thresh_index])
        thresh_locs[col] = n_features_list_new[thresh_index]
    ranks = np.argsort(thresh_locs)
    forest[col,:,:].mean(axis=1)
    for rank,ax in enumerate(axes.flat):
        col = ranks[rank]
        ax.errorbar(n_features_list,forest[col,:,:].mean(axis=1),forest[col,:,:].std(axis=1)/np.sqrt(n_splits),color='red',label=labels[0])
        ax.errorbar(n_features_list,lin[col,:,:].mean(axis=1),lin[col,:,:].std(axis=1)/np.sqrt(n_splits),color='black',label=labels[1])
        if col==0:
            handles, labels = ax.get_legend_handles_labels()
            lg = ax.legend(handles[0:], labels[0:], loc=4, fontsize=16)
            lg.draw_frame(False)
        ax.set_xlim(0.5,20000)
        ax.set_ylim(0,0.8)
        ax.set_yticks(np.linspace(0,0.6,4))
        ax.set_yticklabels([_ for _ in np.linspace(0,0.6,4)],size=20)
        ax.set_xticklabels(n_features_list,size=20)
        ax.set_xscale('log')
        ax.set_title(descriptors[col], size=25)
        ax.plot([0,thresh_locs[col]],[thresholds[col],thresholds[col]],'b--')
        ax.plot([thresh_locs[col],thresh_locs[col]],[0,thresholds[col]],'b--')
        #break
        print('%s: %.1f' % (descriptors[col],thresh_locs[col]))#,thresholds[col])
    fig.text(0.5, 1.00, 'Threshold = %d%%' % (threshold*100), ha='center', size=25)
    fig.text(0.5, -0.025, 'Number of features', ha='center', size=25)
    fig.text(-0.02, 0.5, 'Correlation', va='center', rotation='vertical', size=25);
    plt.tight_layout()

In [ ]:
plot(rs_forest[:,:,:n_splits],rs_lin[:,:,:n_splits],n_features_list,labels=['forest','linear'],threshold=0.8)
#plt.savefig('../../figures/forest_vs_features_%d.eps' % (0.8*100),format='eps',bbox_inches='tight',pad_inches=0.5)